In [16]:
from keras.layers import Dense, Embedding, Input, Subtract, Flatten
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [17]:
games = pd.read_csv("games_season.csv")
tourney = pd.read_csv("games_tourney.csv")

In [18]:
def get_split_point(df, y="score_diff", split=0.8, x=None):

  y_final = df.loc[:,df.columns == y]

  if x is None:
    X_final = df.loc[:,df.columns != y]

  else:
    X_final = df.loc[:,df.columns == x]

  X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2)
  return X_train, y_train, X_test, y_test

In [19]:
X_train_games, X_test_games, y_train_games, y_test_games = get_split_point(games)

In [20]:
X_train_tourney, X_test_tourney, y_train_tourney, y_test_tourney = get_split_point(tourney)

In [21]:
train_cols_games = len(X_train_games.columns)
train_cols_tourney = len(X_train_tourney.columns)

In [46]:
input_tensor_1 = Input(shape=(1,))
input_tensor_2 = Input(shape=(1,))
n_teams = np.unique(X_train_games['team_1']).shape[0]
embed_layer = Embedding(input_dim=n_teams, input_length=1, output_dim=1, name="Team-Strength-Lookup")
embed_tensor = embed_layer(input_tensor)
flatten_tensor = Flatten()(embed_tensor)
team_strength_model = Model(input_tensor, flatten_tensor)

In [47]:
team_1_strength = team_strength_model(input_tensor_1)
team_2_strength = team_strength_model(input_tensor_2)

In [48]:
score_diff = Subtract()([team_1_strength, team_2_strength])

In [51]:
model = Model([input_tensor_1, input_tensor_2], score_diff)
model.compile(optimizer="adam", loss="mean_absolute_error")

In [52]:
input_1 = games["team_1"]
input_2 = games["team_2"]
model.fit([input_1, input_2], games["score_diff"], epochs=1, batch_size=2048, validation_split=0.1, verbose=1)

138/138 [==============================] - 1s 3ms/step - loss: 12.1201 - val_loss: 11.8384


In [53]:
test_input_1 = tourney["team_1"]
test_input_2 = tourney["team_2"]
eval = model.evaluate([test_input_1, test_input_2], tourney["score_diff"], verbose=False)

In [55]:
print("Tournament MAE: {}".format(eval))

Tournament MAE: 11.681591987609863
